In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, KFold

In [ ]:
# check gpu
print(tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
))

tf.__version__

In [ ]:
dftrain = pd.read_csv("data-P1.csv")
dftrain

In [ ]:
dftest = pd.read_csv("data-P2.csv")
dftest

In [ ]:
missing_train = dftrain[dftrain.isna().any(axis=1)]["timestep"].unique()  # dealing with missing data

train_set = dftrain[~dftrain.timestep.isin(missing_train)]
    
train_set

In [ ]:
missing_test = dftest[dftest.isna().any(axis=1)]["timestep"].unique()  # dealing with missing data

test_set = dftest[~dftest.timestep.isin(missing_test)]

test_set

In [ ]:
def df_to_network_in(df: pd.DataFrame, target_present: bool = True):

    n_catch = df["code"].unique().size

    # % Drop info columns
    df = df.drop(["code", "timestep"], axis=1)  

    if target_present:
        # check if 'bias' is already the last column
        if df.columns[-1] != "bias":
            columns = [col for col in df.columns if col != "bias"]
            columns.append("bias")
            df = df[columns]
        
        # convert to numpy array
        data = df.to_numpy()[..., :-1]
        target = df.to_numpy()[..., -1]
        target = target.reshape(-1, n_catch)

    else:
        data = df.to_numpy()
        target = None

    # % Normalize
    data = RobustScaler().fit_transform(data) 
    data = data.reshape(-1, n_catch, data.shape[-1])

    return data, target

In [ ]:
train, target = df_to_network_in(train_set)

train.shape, target.shape

In [ ]:
test, target_test = df_to_network_in(test_set)

test.shape, target_test.shape

In [ ]:
strategy = tf.distribute.MirroredStrategy()
strategy

In [ ]:
def lstm_net(input_shape):

    net = tf.keras.Sequential()

    net.add(Bidirectional(LSTM(64, input_shape=input_shape, return_sequences=True)))
    net.add(Dropout(0.2))
    net.add(Bidirectional(LSTM(64, return_sequences=True)))
    net.add(Dropout(0.2))
    net.add(Dense(16, activation='selu'))
    net.add(Dense(1))

    return net

In [ ]:
EPOCH = 200
BATCH_SIZE = 512
K_FOLD = 5

net_path = "nets/net"
test_preds = []
history = []

In [ ]:
# def nse_loss(y_true, y_pred):
#     numerator = tf.reduce_sum((y_true - y_pred)**2)
#     denominator = tf.reduce_sum((y_true - tf.reduce_mean(y_true))**2)
#     return numerator / denominator

In [ ]:
with strategy.scope():    
    
    kf = KFold(n_splits=K_FOLD, shuffle=True)
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, target)):  # Cross Validation Training

        print(f'</> Training Fold {fold + 1}...')

        X_train, X_valid = train[train_idx], train[test_idx]
        y_train, y_valid = target[train_idx], target[test_idx]

        net = lstm_net(train.shape[-2:])
        net.compile(optimizer="adam", loss="mse")

        scheduler = ExponentialDecay(1e-3, 100*((train.shape[0]*0.8)/BATCH_SIZE), 1e-5)
        lr = LearningRateScheduler(scheduler)
        
        cp = tf.keras.callbacks.ModelCheckpoint(
            filepath=f"{net_path}_fold{fold + 1}",
            save_weights_only=True,
            mode='min',
            save_best_only=True)
    
        history.append(net.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=EPOCH, batch_size=BATCH_SIZE, callbacks=[lr,cp]))

In [ ]:
history

In [ ]:
for hist in history:
    tl = hist.history["loss"]
    vl = hist.history["val_loss"]
    plt.plot(range(len(tl)), tl, vl)
    # plt.ylim([0,5])
    plt.show()

In [ ]:
nets = []
for fold in range(K_FOLD):
    net = lstm_net(train.shape[-2:])
    net.load_weights(f"{net_path}_fold{fold + 1}")
    nets.append(net)

In [ ]:
ypred = np.mean([net.predict(train) for net in nets], axis=0)

In [ ]:
plt.plot(range(ypred.size),target.flatten(), ypred.flatten())

In [ ]:
for loss in [mean_absolute_error, mean_squared_error]:
    print(f"{loss.__name__}: not corrected {loss(np.zeros(target.flatten().shape),target.flatten())}, corrected {loss(ypred.flatten(),target.flatten())}")

In [ ]:
df_correct = pd.DataFrame({"code": train_set["code"], "timestep": train_set["timestep"], "bias": ypred.flatten()})
df_correct.to_csv("data-corrected-P1.csv", index=False)

In [ ]:
ypred_test = np.mean([net.predict(test) for net in nets], axis=0)

In [ ]:
plt.plot(range(ypred_test.size),target_test.flatten(), ypred_test.flatten())

In [ ]:
for loss in [mean_absolute_error, mean_squared_error]:
    print(f"{loss.__name__}: not corrected {loss(np.zeros(target_test.flatten().shape),target_test.flatten())}, corrected {loss(ypred_test.flatten(),target_test.flatten())}")

In [ ]:
df_correct = pd.DataFrame({"code": test_set["code"], "timestep": test_set["timestep"], "bias": ypred_test.flatten()})
df_correct.to_csv("data-corrected-P2.csv", index=False)